# Bag of Words Meets Bags of Popcorn


### 사용되는 파일

* **labeledTrainData** - 분류된 Training Set. Tab구분자로 되어 있고 컬럼의 헤더를 가진다. ID, Sentiment, Review를 25000건 포함
* **testData** - Test Set. Tab구분자이고 25000건의 리뷰 정보를 담고 있음 
* **unlabeledTrainData** - 분류되지 않은 확장 Training Set. Id와 Review 50000건을 포함 (Sentiment 없음)
* **sampleSubmission** - kaggle 제출용. Comma 구분자로 되어 있음

### Data fields
* **id** - 각 리뷰의 Unique한 ID
* **sentiment** - 리뷰의 정서(평점), 좋은 리뷰는 1 / 나쁜 리뷰는 0
* **review** - Review의 텍스트

## Reading the Data



In [2]:
import pandas as pd

In [3]:
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
# Arg 1 - 파일 경로
# Arg delimiter - 구분자
# Arg quoting - 3은 Quote를 허용하지 않음  (예) "Mad Max II" -> \"Mad Max II\")
train = pd.read_csv("input/labeledTrainData.tsv", delimiter="\t", quoting=3)
print(train.shape)
train.head()

(25000, 3)


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [4]:
#Review 살펴보기
#print(train["review"][0])

## Data Cleaning and Text Preprocessing

인터넷에서 가져온 데이터는 HTML 태그가 섞여 있는 형태일 수 있음. HTML을 제거하는 작업을 진행

Beautiful Soup 라이브러리를 사용하여 제거할 수 있다

> **_Beautiful Soup_**
HTML 형태의 텍스트를 간단히(!) 처리하기 위한 라이브러리
* **get_text()** - 텍스트만을 추출
* **prettify()** - 코드 정렬
* **find_all()** - querySelectorAll과 비슷한 개념. 해당 태그들을 텍스트 리스트 형태로 가져옴.
* **title** - 타이틀 태그 의미, **title.name** - 타이틀의 태그 이름, **title.string** - 타이틀의 텍스트 (태그 직접접근 개념)

In [5]:
#만약 Beautiful Soup라이브러리 설치가 되어 있지 않다면 아래의 코드를 실행
#!pip install BeautifulSoup4

In [6]:
from bs4 import BeautifulSoup             

# 하나의 Review를 BeautifulSoup object로 초기화
example1 = BeautifulSoup(train["review"][9])  

# 원본 Review와 가공된 Review 비교
# HTML 태그가 제거된 것을 확인할 수 있음
print(train["review"][9])
print(example1.get_text())

"<br /><br />This movie is full of references. Like \"Mad Max II\", \"The wild one\" and many others. The ladybug´s face it´s a clear reference (or tribute) to Peter Lorre. This movie is a masterpiece. We´ll talk much more about in the future."
"This movie is full of references. Like \"Mad Max II\", \"The wild one\" and many others. The ladybug´s face it´s a clear reference (or tribute) to Peter Lorre. This movie is a masterpiece. We´ll talk much more about in the future."


C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


이 예제에서는 숫자와 punctuation(구두점, !!! :- 와 같은 문자를 제거한다) 

그 것은 _Beautiful Soup_ 로 처리할 수 없음. Regular Expression(re라 불리는)을 이용하여 제거할 수 있다

In [7]:
# https://docs.python.org/2/library/re.html#
import re

# Args 1 - Regular Expression. 밑의 예제는 알파벳이 아닌 모든 문자를 의미
# Args 2 - Replace할 텍스트. 밑의 예제는 빈 값(스페이스 한 칸)을 의미
# Args 3 - 대상이 되는 텍스트
letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      example1.get_text() )  # The text to search
print(letters_only)

 This movie is full of references  Like   Mad Max II      The wild one   and many others  The ladybug s face it s a clear reference  or tribute  to Peter Lorre  This movie is a masterpiece  We ll talk much more about in the future  


이제 모든 알파벳을 소문자로 치환하고 단어 단위로 모두 나눌거다. 이 때 특정 데이터를 나누는 작업을 Tokenization이라고 한다

In [8]:
lower_case = letters_only.lower()        # 소문자로 바꾸기
words = lower_case.split()               # 단어로 단위로 나누기

자주 발생하지만 많은 의미를 담고 있지 않는 단어들을 어떻게 할지 결정해야 한다

이러한 단어를 "stop words"라고 하는데 영어에서는 "a", "is", "is", "the" 등이 해당된다.

이러한 단어들은 편리하게 **_Natural Language Tookit(NLTK)_** 에 등록되어 있다

In [9]:
# nltk를 다운로드 받는다
import nltk
nltk.download('stopwords')  # Download text data sets, including stop words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [10]:
from nltk.corpus import stopwords # stop word 리스트를 로드 
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [11]:
# words로 부터 stopwords를 제거한다
words = [w for w in words if not w in stopwords.words("english")]
print(words)

['movie', 'full', 'references', 'like', 'mad', 'max', 'ii', 'wild', 'one', 'many', 'others', 'ladybug', 'face', 'clear', 'reference', 'tribute', 'peter', 'lorre', 'movie', 'masterpiece', 'talk', 'much', 'future']


이제까지 했던 작업들을 재사용하기 위해 하나의 함수로 정의한다

In [12]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   


In [13]:
clean_review = review_to_words( train["review"][9] )
print(clean_review)

movie full references like mad max ii wild one many others ladybug face clear reference tribute peter lorre movie masterpiece talk much future


C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


해당 함수를 모든 Review(25000건)에 대해 호출하기 전에 오래 걸리니까 진행 상태를 보기 위한 작업을 한다 

In [16]:
!pip install tqdm

In [17]:
from tqdm import tqdm
tqdm.pandas(desc="Review To Words..")

모든 review(250000건)에 동일한 작업을 하자. 컴퓨터에 따라 오래 걸릴 수도 있다

In [18]:
train["review"].progress_apply(review_to_words)
print(train["review"][9])

Review To Words..:   0%|          | 0/25000 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
Review To Words..: 100%|██████████| 25000/25000 [00:45<00:00, 545.23it/s]


"<br /><br />This movie is full of references. Like \"Mad Max II\", \"The wild one\" and many others. The ladybug´s face it´s a clear reference (or tribute) to Peter Lorre. This movie is a masterpiece. We´ll talk much more about in the future."


## Creating Features from a Bag of Words (Using scikit-learn)

이제 Training Set의 Review가 깔끔하게 정리됐다. 이제 머신러닝 모델이 이해할 수 있도록 numeric의 데이터로 변환해야 한다. 이를 Bag of Words라고 한다.

Bag of Words 모델은 문서로부터 vocabulary를 학습한다. 그리고 모델은 단어의 수를 카운팅한다. 

우리가 사용하는 IMDB 데이터는 매우 많은 리뷰가 있고 많은 vocabulary가 존재하게 된다. 최대 vocabulary를 자주 사용되는 단어 5000개로 제한한다.(기억해라 stop words는 이미 제거된 상태이다) 

In [19]:
clean_train_reviews = train["review"]

from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

In [20]:
# 25000개의 Review와 5000개의 vocabulary가 있다 
print(train_data_features.shape)

(25000, 5000)


> **_CountVectorizer_** 는 preprocessing, tokenization 그리고 stop word 제거 작업을 해주는 옵션이 있다. 지금은 우리가 만든 기능들을 쓰기 위해 None으로 했다. 

In [25]:
# 사용되는 Vocabulary 확인
vocab = vectorizer.get_feature_names()
print(vocab[:100])

['00', '000', '10', '100', '11', '12', '13', '13th', '14', '15', '16', '17', '18', '1930', '1930s', '1933', '1940', '1950', '1950s', '1960', '1960s', '1968', '1970', '1970s', '1972', '1973', '1980', '1980s', '1983', '1984', '1987', '1990', '1996', '1997', '1999', '1st', '20', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '20th', '24', '25', '2nd', '30', '30s', '35', '3d', '3rd', '40', '45', '50', '50s', '60', '60s', '70', '70s', '80', '80s', '90', '90s', '99', 'abandoned', 'abc', 'abilities', 'ability', 'able', 'about', 'above', 'abraham', 'absence', 'absent', 'absolute', 'absolutely', 'absurd', 'abuse', 'abusive', 'abysmal', 'academy', 'accent', 'accents', 'accept', 'acceptable', 'accepted', 'access', 'accident', 'accidentally', 'accompanied', 'accomplished', 'according', 'account', 'accurate', 'accused', 'achieve', 'achieved']


In [26]:
# 각 단어의 카운트를 보고싶다면 아래 주석을 풀고 싷행
import numpy as np

# Sum up the counts of each vocabulary word
# dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
#for tag, count in zip(vocab, dist):
    #print(count, tag)

## Random Forest

이제 우리는 Numeric Training Feature를 가지고 학습할 수 있다. 이 내용은 preprocessing을 위한 프로젝트이므로 Random Forest는 깊게 다루지 않는다.

In [27]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

In [28]:
# Read the test data
test = pd.read_csv("input/testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print(test.shape)

# Create an empty list and append the clean reviews one by one
test["review"].progress_apply(review_to_words)
clean_test_reviews = test["review"]

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "output/Bag_of_Words_model.csv", index=False, quoting=3 )

(25000, 2)


Review To Words..:   0%|          | 0/25000 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
Review To Words..: 100%|██████████| 25000/25000 [00:44<00:00, 557.56it/s]
